In [1]:
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs
using Statistics
using Plots, ColorSchemes, LaTeXStrings

In [2]:
T = 20
N = 10^2
k = 3
nsamples = 10^3;

In [14]:
# seed = 7111
seed = 1
gg = erdos_renyi(N, k/N; seed)
@show length(connected_components(gg))
g = IndexedGraph(gg)

w = map(1:N) do i
    ei = inedges(g, i)
    J = -1
    fill(HomogeneousVoterFactor(J), T + 1)
end;

length(connected_components(gg)) = 5


In [21]:
m0 = [0.7, 0.8, 0.9] |> reverse

samplers = map(eachindex(m0)) do a
    m = m0[a]
    bp = mpbp(IndexedBiDiGraph(gg), w, fill(2,N), T)
    for i in eachindex(bp.ϕ)
        bp.ϕ[i][1] .= [(1+m)/2,(1-m)/2]
    end
    sms = sample(bp, nsamples; showprogress=false)
    println("Finished magnetiz $m ($a/$(length(m0)))")
    sms
end;

MethodError: MethodError: reducing over an empty collection is not allowed; consider supplying `init` to the reducer

In [16]:
spin(x, i) = 3 - 2x
magnetiz(sms) = mean(getproperty.(mm, :val) for mm in means(spin, sms))
m = [magnetiz(sms) for sms in samplers];

UndefVarError: UndefVarError: `samplers` not defined

In [17]:
cg = cgrad(:matter, length(m0)+1, categorical=true)
pl = plot(xlabel=L"t", ylabel=L"m", legend=:outertopright)
for a in eachindex(samplers)
    plot!(pl, 0:T, m[a], label="m⁰=$(m0[a])", c=cg[a+1])
end
pl

UndefVarError: UndefVarError: `samplers` not defined

In [18]:
spin(x) = spin(x, 0)

e = map(samplers) do sms
    pm = pair_marginals_alternate(sms; showprogress=false)
    ee = map(pm) do bij
        expectation.(spin, bij)
    end
    mean(ee)
end;

UndefVarError: UndefVarError: `samplers` not defined

In [19]:
cg = cgrad(:matter, length(m0)+1, categorical=true)
ple = plot(xlabel=L"t", ylabel=L"c", legend=:outertopright)
for a in eachindex(samplers)
    plot!(ple, 1:T, -ne(gg)/N*e[a], label="m⁰=$(m0[a])", c=cg[a+1])
end
ple

UndefVarError: UndefVarError: `samplers` not defined

In [20]:
# using DelimitedFiles
# open("energy.txt", "w") do io
#     writedlm(io, reduce(hcat, e))
# end
# open("magnetiz.txt", "w") do io
#     writedlm(io, reduce(hcat, m))
# end

## MPBP

In [10]:
sms = samplers[1]
bp = sms.bp;

UndefVarError: UndefVarError: `samplers` not defined

In [11]:
cb = CB_BP(bp; showprogress=false)
maxiter = 10
svd_trunc = TruncBond(5)
tol = 1e-5
iterate!(bp; maxiter, svd_trunc, cb, tol);

UndefVarError: UndefVarError: `bp` not defined